# **Project results on the freesurfer surfaces**
This notebook will guide you through the step necessary to project fMRI results on the surfaces that were generated with Freesurfer.

### Initiation - Variables and paths

In [ ]:
# what subject?
SUBJ=Danny

# where will the result appear?
OUT=/Users/chris/Desktop/Epi2Surf_${SUBJ}
mkdir -p $OUT

# where can we find the epi space the results are in?
# This is usually the reference from the `manual-masks' folder that is used for preprocessing and modelfit in NHP-BIDS.

# Danny
EPI_ROOT=/Users/chris/Documents/MRI_ANALYSIS/NHP-BIDS/manual-masks/sub-danny/func/
EPI=${EPI_ROOT}/sub-danny_ref_func_res-1x1x1.nii.gz
EPI_MASK=${EPI_ROOT}/sub-danny_ref_func_mask_res-1x1x1.nii.gz

# Eddy
#EPI_ROOT=/Users/chris/Documents/MRI_ANALYSIS/NHP-BIDS/manual-masks/final/sub-eddy/ses-20170607b/func/
#EPI=${EPI_ROOT}/ref_func_undist_inData_al_fnirt.nii.gz
#EPI_MASK=${EPI_ROOT}/HiRes_to_T1_mean.nii_shadowreg_Eddy_brainmask.nii.gz

# where is the anatomical you'd like to register to?
# $SUBJECTS_DIR should point to the Freesurfer subjects directory. Adjust your bashrc to export this if it doesn't do so already.
T1=${SUBJECTS_DIR}/${SUBJ}/mri/brainmask.mgz
# and the corresponding white matter segmentation?
WM=${SUBJECTS_DIR}/${SUBJ}/mri/wm.mgz

# where's this notebook on the disk?
NOTEBOOK_PATH=/Users/chris/Documents/MRI_ANALYSIS/NHP-Freesurfer/Notebooks # used to identify example data in a subfolder

In [ ]:
# Mask the epi that is used as a reference in functional analysis to extract the brain
fslmaths ${EPI} -mas ${EPI_MASK} ${OUT}/epi_brain.nii.gz
EPI_BRAIN=${OUT}/epi_brain.nii.gz

## Flirt registration of epi to T1
The T1 you pick here is the header adjusted one that formed the based of the surface generation in Freesurfer. We do not have to adjust the header of the epi, because flirt will generally be able to take of the scaling. If this fails for some reason you can still do it manually with `3drefit -xdel 2.0 -ydel 2.0 -zdel 2.0 -keepcen <epi>.nii.gz`. Note that we set voxel sizes to 2 mm in the header in this case. The reason is that for the T1 we had 0.5 mm voxels and adjusted the header to 1 mm, a factor of 2. Our pre-processed epi's have 1 mm voxels, so applying the same factor of 2 means the header info should state that the voxels are 2 mm isotropic.

Be aware that, when you first adjust the header and then do flirt, the final registration matrix will not include this scaling and expect header-adjusted input. Any result you want to warp to the surface will thus first have to be adjusted. You don't have to do this is if the flirt is performed on the non-adjusted epi. 

NB1! One thing to carefully check is whether the x-direction of the voxel order matches between the epi and T1 because if it doesn't we will see left/right flips. With some of our older standard epi's this needs correction (later I corrected it in the reference file for pre-processing). There's a script that does that for you called `swap_xdir_voxels.sh`. You can find it in the `bin` folder of the `Process-NHP-MRI` repository (https://github.com/VisionandCognition/Process-NHP-MRI)

NB2! flirt works significantly better if you include the white matter segmentation (`-wmseg`) of the T1 and phase encoding direction (`-pedir`) of the epi. Check the documentation to find how to code this. If you used Chris Klink's standard epi sequence, your `-pedir` is `-2`.

In [ ]:
# get the brain and white matter volumes from freesurfer & convert to nifti
mri_convert ${T1} ${OUT}/brain.nii.gz
mri_convert ${WM} ${OUT}/wm.nii.gz
# calculate the registration
flirt -ref ${OUT}/brain.nii.gz -wmseg ${OUT}/wm.nii.gz -in ${EPI_BRAIN} -out ${OUT}/epi2anat.nii.gz -omat ${OUT}/epi2anat.mat -pedir -2   
# check whether nonlinear registration improves on this result (could be a mess as well)
# fnirt --ref ${OUT}/brain.nii.gz --in ${EPI_BRAIN} --aff ${OUT}/epi2anat.mat --iout ${OUT}/epi2anat_fnirt.nii.gz --inmask ${EPI_MASK} 

## Create the tkregister matrix
Now we will use the flirt registration matrix to create a registration matrix in freesurfer format. To get there, we can use the freesurfer program `tkregister` that allows manual registration between 2 volumes. We will inititate it with the flirt matrix and check whether registration is good. If not, you can make manual adjustments, but flirt probably does a better job than manual attempts. Within `tkregister` you can check the alignment of the two volumes by clicking `compare`. Save the registration matrix as `reg.fsl.dat`.

NB! Freesurfer's `tktools` do currently not work on Mac OS X Catalina. This step should be run on a (virtual) machine with a different OX.

![tkregister](pics/tkregister.gif "tkregister")

In [ ]:
# run the tkregister registration initated with the flirt transform
tkregister2 --mov ${EPI_BRAIN} --targ ${OUT}/brain.nii.gz --fsl ${OUT}/epi2anat.mat --reg ${OUT}/reg.fsl.dat --s ${SUBJ}

In [ ]:
tkReg=${OUT}/reg.fsl.dat 

## Convert volumes to surface representation

In [ ]:
# These (re)definitions allow to do this part independent of previous steps
SUBJ=${SUBJ} 
tkReg=${OUT}/reg.fsl.dat 

# this is one of the visual localizers from our curve tracing experiment with Danny >> left side stimulus, so look at the right hemisphere for activations
RES_LC=${NOTEBOOK_PATH}/example_data/tstat1.nii.gz 

HEMI=(lh rh) # allow looping over hemispheres when calculating volume to surface transformations

We can now convert the statistical volumes to surface representations using the `mri_vol2surf` command. Since this essentially brings a 3d result to 2d there is a choice to be made on how/where to sample. The `--projfrac` tells the command where between the WM/GM border (`--projfract 0`) and the pial surface (`projfract 1`) to get the data. These fractions can also be negative (going into the WM) or higher than 1 (beyond the pial surface). Alternatively, you can average along the normal between WM/GM border and pial surface using `--projfrac-avg min max stepsize` or the maximum by using `--projfrac-max min max stepsize`.

In [ ]:
# look at the mri_vol2surf documentation for more info
# mri_vol2surf --help

In [ ]:
# create example surface plot
for xh in ${HEMI[@]}; do
    # white matter surface
    mri_vol2surf --trgsubject ${SUBJ} --src ${RES_LC} --out ${OUT}/${xh}.targ1_wm.w --out_type paint --projfrac 0 --srcreg ${tkReg} --hemi ${xh}
    # midcortical
    mri_vol2surf --trgsubject ${SUBJ} --src ${RES_LC} --out ${OUT}/${xh}.targ1_midcort.w --out_type paint --projfrac 0.5 --srcreg ${tkReg} --hemi ${xh}
    # average on normal
    mri_vol2surf --trgsubject ${SUBJ} --src ${RES_LC} --out ${OUT}/${xh}.targ1_avg.w --out_type paint --projfrac-avg 0 1 0.2 --srcreg ${tkReg} --hemi ${xh}
done

In [ ]:
# show in tksurfer
xh=rh # switch hemisphere easily
tksurfer ${SUBJ} ${xh} graymid -patch full.patch.flat -overlay ${OUT}/${xh}.targ1_avg.w -overlay-reg ${tkReg}

Results will look somewhat like this:
(note that these images are from a different example/hemisphere)

![tstat_inflated](pics/tstat_inflated.png "tstat_inflated") ![tstat_inflated](pics/tstat_inflated_legend.png "tstat_inflated_legend")

(Left: full.patch, Right: occip.patch):

![tstat_fullflat](pics/tstat_fullflat.png "tstat_fullflat") ![tstat_occipflat](pics/tstat_occipflat.png "tstat_occipflat")

## Project a retinotopic map on the surface

Freesurfer wants the polar angle map split up in vector form where the X and Y component each get a map. In polar coordinates, a point in space can be described by a `radius` and `angle`. In cartesian coordinates, the same point has an `X` and `Y` value. A conversion from polar angle to Cartesian coordinates can be done by calculating the `cosine` and `sine` components that form X and Y. Since both are equally scaled by the radius (eccentricity) we can ignore this element when converting polar angle maps (set it to one). 

![Polar_vector](pics/Polar_vector.png "polar_vector")

When these X and Y components are calculated directly from the bold response with a GLM (for phase-encoded stimuli) they essentially represent the beta values for a fitted sine and cosine function respectively. When calculated using fast Fourier transforms instead of a GLM, they are the real and imaginary parts of the complex number output.

In essence, all that matters in describing the polar angle is the ratio of X to Y, or the vector that is described by X and Y.

After pRF mapping, we get an optimal polar angle for each voxel. Converting these to a vector representation by calculating the cosine and sine of this angle **in radians** gives us two maps with values ranging from -1 to 1 (since we assume a radius of 1). To have our Freesurfer polar angle maps match the volumetric polar angle maps from the pRF analysis (calculated with `analyzePRF`) we need to multiply the cosine component with -1. Expressing the polar angle as a vector this way has several advantages. 

- It easily allows using zeros to mask out voxels (tksurfer has an option to not plot zeros)
- It allows averaging of the individual X and Y components across repetitions or along the surface normal. This is not possible with polar angles since they are circular (e.g., averaging the very similar angles of 359 and 1 degrees would give the completley opposite result of 180 degrees) 

In [ ]:
# redefining helps if we only want to do this part
SUBJ=${SUBJ} 
tkreg=${OUT}/reg.fsl.dat 

# link up some retinotopic mapping results (these are unthresholded)
#POL=${NOTEBOOK_PATH}/example_data/POL.nii.gz # prf polar angle map
#ECC=${NOTEBOOK_PATH}/example_data/ECC.nii.gz # prf eccentricity map
#RFS=${NOTEBOOK_PATH}/example_data/RFS.nii.gz # prf RF size map
#R2=${NOTEBOOK_PATH}/example_data/R2.nii.gz # prf R2 map, can be used for masking

# subject specific
POL=${NOTEBOOK_PATH}/RetMaps/${SUBJ}/POL.nii.gz # prf polar angle map
ECC=${NOTEBOOK_PATH}/RetMaps/${SUBJ}/ECC.nii.gz # prf eccentricity map
RFS=${NOTEBOOK_PATH}/RetMaps/${SUBJ}/RFS.nii.gz # prf RF size map
R2=${NOTEBOOK_PATH}/RetMaps/${SUBJ}/R2.nii.gz # prf R2 map, can be used for masking

# allow looping over hemispheres
HEMI=(lh rh) 

In [ ]:
# convert to radians
fslmaths ${POL} -mul 0.01745329252 ${OUT}/angle_fs.nii.gz # radians (multiply with pi/180)

# create a volumetric map so we can check alignment later on if needed go back to improve registration
flirt -ref ${OUT}/brain.nii.gz -in ${POL} -out ${OUT}/angle_reg.nii.gz -applyxfm -init ${OUT}/epi2anat.mat -interp nearestneighbour
mri_convert ${OUT}/angle_reg.nii.gz ${OUT}/angle_reg.mgz

# create real (-COS) and imaginary (SIN) components for the polar angle
fslmaths ${OUT}/angle_fs.nii.gz -cos -mul -1 ${OUT}/angle_real.nii.gz
fslmaths ${OUT}/angle_fs.nii.gz -sin ${OUT}/angle_imag.nii.gz

Before looking at surface renderings, let's first inspect this in volumetric form to see how well the registered pRF results align with the anatomy.

In [ ]:
# Check volumetric maps
freeview -v ${SUBJECTS_DIR}/${SUBJ}/mri/brainmask.mgz -v ${OUT}/epi2anat.nii.gz -v ${OUT}/angle_reg.mgz \
    -f ${SUBJECTS_DIR}/${SUBJ}/surf/lh.smoothpial:edgecolor=red -f ${SUBJECTS_DIR}/${SUBJ}/surf/rh.smoothpial:edgecolor=red \
    -f ${SUBJECTS_DIR}/${SUBJ}/surf/lh.smoothwm:edgecolor=yellow -f ${SUBJECTS_DIR}/${SUBJ}/surf/rh.smoothwm:edgecolor=yellow &

In [ ]:
# where to sample for the creation of surface values
n_min=-1 # go slightly into WM
n_max=1 # stop before pial border to avoid crossing sulcus
n_step=0.1 # stepsize

# Create surfaces using unthresholded pRF maps
for xh in ${HEMI[@]}; do
    mri_vol2surf --trgsubject ${SUBJ} --src ${OUT}/angle_fs.nii.gz --out ${OUT}/${xh}.angle_fs.w \
        --out_type paint --projfrac 0.5 --srcreg ${OUT}/reg.fsl.dat --hemi ${xh}
    mri_vol2surf --trgsubject ${SUBJ} --src ${OUT}/angle_real.nii.gz --out ${OUT}/${xh}.angle_real.w \
        --out_type paint --projfrac-avg ${n_min} ${n_max} ${n_step} --srcreg ${OUT}/reg.fsl.dat --hemi ${xh}
    mri_vol2surf --trgsubject ${SUBJ} --src ${OUT}/angle_imag.nii.gz --out ${OUT}/${xh}.angle_imag.w \
        --out_type paint --projfrac-avg ${n_min} ${n_max} ${n_step} --srcreg ${OUT}/reg.fsl.dat --hemi ${xh}
    mri_vol2surf --trgsubject ${SUBJ} --src ${R2} --out ${OUT}/${xh}.R2.w --out_type paint \
        --projfrac-avg ${n_min} ${n_max} ${n_step} --srcreg ${OUT}/reg.fsl.dat --hemi ${xh}
done

Displaying phase encoded data in `tksurfer` is a little different from displaying statistical maps on the surface. You will have to load the real and imaginary components to overlay 1 and 2 respectively (with the associated registration matrix). Then you go to `View > Configure > Overlay` to set the color scale to `RYGB wheel` and the display options to `complex`. The thresholds to something low like `0.001` (both min and max; make sure to press enter after changing these values). You can tick the box to `ignore zeros in histogram` which will remove the masked voxels. Press `Apply`. If you want to directly see the polar angle values under you mouse cursor you can activate this with `View > Information > Angle`. You can cycle between view types with the brain icons (Main/Inflated/White/Pial/Original surfaces).

![tksurfer_retmap](pics/tksurfer_retmap.png "tksurfer_retmap")


In [ ]:
# inspect result in tksurfer (will be very confetti-like dues to all poorly fitted voxels not getting filtered)
xh=rh
tksurfer ${SUBJ} ${xh} inflated -patch full.patch.flat -overlay ${OUT}/${xh}.angle_real.w -overlay-reg ${tkreg}

These unthresholded maps don't make a ton of sense, so we'd better use the R2-map to threshold the results and show only voxels with a decent fit.

In [ ]:
# Threshold the map at R2 value
R2_thr=5

In [ ]:
# mask the volumetric maps by R2 threshold
fslmaths ${R2} -thr ${R2_thr} -bin ${OUT}/rmask.nii.gz
fslmaths ${OUT}/angle_real.nii.gz -mas ${OUT}/rmask.nii.gz ${OUT}/angle_real_thr${R2_thr}.nii.gz
fslmaths ${OUT}/angle_imag.nii.gz -mas ${OUT}/rmask.nii.gz ${OUT}/angle_imag_thr${R2_thr}.nii.gz
fslmaths ${ECC} -mas ${OUT}/rmask.nii.gz ${OUT}/ECC_thr${R2_thr}.nii.gz

In [ ]:
# convert to surfaces
for xh in ${HEMI[@]}; do
    mri_vol2surf --trgsubject ${SUBJ} --src ${OUT}/angle_real_thr${R2_thr}.nii.gz --out ${OUT}/${xh}.angle_real_thr${R2_thr}.w \
        --out_type paint --projfrac-avg ${n_min} ${n_max} ${n_step} --srcreg ${tkreg} --hemi ${xh}
    mri_vol2surf --trgsubject ${SUBJ} --src ${OUT}/angle_imag_thr${R2_thr}.nii.gz --out ${OUT}/${xh}.angle_imag_thr${R2_thr}.w \
        --out_type paint --projfrac-avg ${n_min} ${n_max} ${n_step} --srcreg ${tkreg} --hemi ${xh}
    mri_vol2surf --trgsubject ${SUBJ} --src ${OUT}/ECC_thr${R2_thr}.nii.gz --out ${OUT}/${xh}.ECC_thr${R2_thr}.w \
        --out_type paint --projfrac-avg ${n_min} ${n_max} ${n_step} --srcreg ${tkreg} --hemi ${xh}
done

In [ ]:
# Look at the result
xh=lh
tksurfer ${SUBJ} ${xh} inflated -patch full.patch.flat -overlay ${OUT}/${xh}.angle_real_thr${R2_thr}.w -overlay-reg ${tkreg}

In [ ]:
# Look at the result
xh=lh
tksurfer ${SUBJ} ${xh} inflated -patch full.patch.flat -overlay ${OUT}/${xh}.ECC_thr${R2_thr}.w -overlay-reg ${tkreg}